<a href="https://colab.research.google.com/github/siriusted/gym-dssat-notebooks/blob/master/First_steps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# First steps with Gym-dssat

In this tutorial, we will show how to simply interact with the gym-dssat environment.

First, we need to install gym-dssat. This is done in the following code cell. 


## Installation

In [ ]:
!echo "deb [ arch=amd64 ] https://raw.githubusercontent.com/pdidev/repo/ubuntu bionic main" | sudo tee /etc/apt/sources.list.d/pdi.list > /dev/null
!wget -O /etc/apt/trusted.gpg.d/pdidev-archive-keyring.gpg https://raw.githubusercontent.com/pdidev/repo/ubuntu/pdidev-archive-keyring.gpg
!chmod a+r /etc/apt/trusted.gpg.d/pdidev-archive-keyring.gpg /etc/apt/sources.list.d/pdi.list
!apt update &> /dev/null
!apt install pdidev-archive-keyring libpdi-dev &> /dev/null

!wget  http://gac.udc.es/~emilioj/bionic.tgz
!tar -xf bionic.tgz
!cd bionic/ && apt install `find . -name "*.deb"` &> /dev/null

!pip install -U PyYAML &> /dev/null

# add newly installed libraries to path
import sys
sys.path.append('/opt/gym_dssat_pdi/lib/python3.7/site-packages')

Now, we're all set up to use gym-dssat!



We will interact with the environment using two hardcoded policies:

- A null policy
- An expert policy

and display results of their evaluation.

To facilitate this, we define the following helper functions:



In [ ]:
def null_policy(obs):
    '''
    Do not apply any fertilizer
    '''
    return { "anfer": 0}

def expert_policy(obs):
    '''
    Return fertilization amount based on internal map
    and day after planting feature in observation.
    '''
    fertilization_map = {
        40: 27,
        45: 35,
        80: 54,
    }

    amount = fertilization_map.get(obs['dap'], 0)
    
    return { "anfer": amount }

Here, we instantiate our gym-dssat environment. We will use the fertilization mode...

In [ ]:
import gym
import gym_dssat_pdi

env_args = {
  'run_dssat_location': '/opt/dssat_pdi/run_dssat',
  'mode': 'fertilization',
  'seed': 123,
  'random_weather': True,
}

env = gym.make('GymDssatPdi-v0', **env_args)

Then, we define an evaluation and a plotting function below which is used to evaluate and visualize the results of our policies...

In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

def evaluate(policy, n_episodes=10):
    returns = [None] * n_episodes

    for episode in range(n_episodes):
        done = False
        obs = env.reset()
        ep_return = 0

        while not done:
            action = policy(obs)
            obs, reward, done, _ = env.step(action)
            if reward is not None:
                ep_return += reward

        returns[episode] = ep_return

    return returns


def plot_results(data):
    data_dict = {}
    for label, returns in data:
        data_dict[label] = returns
    df = pd.DataFrame(data_dict)
    
    ax = sns.boxplot(data=df)
    ax.set_xlabel("policy")
    ax.set_ylabel("evaluation output")
    plt.show()

Evaluate null policy

In [ ]:
print('Evaluating null policy...')
null_returns = evaluate(null_policy)
print('Done')

Evaluate expert policy

In [ ]:
print('Evaluating expert policy...')
expert_returns = evaluate(expert_policy)
print('Done')



Finally, we display the results...

In [ ]:
results = [('null', null_returns), ('expert', expert_returns)]

plot_results(results)

And, there you have it! Now, you can go ahead and keep playing with gym-dssat by editing and re-running the code cells above.